In [1]:
%reload_ext autoreload
%autoreload 2

# 单个文件overfitting

In [2]:
from models import backbones
from train import train
import pickle
from pathlib import Path
import io
import numpy as np
import torch
from skimage import io, transform
from models.p2m import P2MModel
from utils.mesh import Ellipsoid
from torchvision.transforms import Normalize
from models.losses.p2m import P2MLoss
from easydict import EasyDict as edict
from utils.visualization import visualize_occupancy, visualize_pointcloud, visualize_mesh


config  = {
    "experiment_name": "template",
    "device": "cuda",
    "learning_rate": 0.0001,
    "batch_size": 8,
    "max_epochs": 100,
    "validate_every_n": 50,
    "print_every_n": 10,
    "loss": edict({"weights": {"normal": 1.6e-4,
        "edge": 0.3,
        "laplace": 0.5,
        "move": 0.1,
        "constant": 1.,
        "chamfer": [1., 1., 1.],
        "chamfer_opposite": 1.,
        "reconst": 0.,} 
    }),
}

# data = pickle.load(open(Path("/root/autodl-tmp/Pixel2Mesh_original-pytorch/datasets/data/shapenet/data_tf/04256520/1a4a8592046253ab5ff61a3a2a0e2484/rendering/01.dat"), 'rb'), encoding="latin1")
# data = pickle.load(open(Path("/root/autodl-tmp/Pixel2Mesh_original-pytorch/datasets/data/shapenet/data_tf/02691156/1a04e3eab45ca15dd86060f189eb133/rendering/00.dat"), 'rb'), encoding="latin1")
data = pickle.load(open(Path("/root/autodl-tmp/Pixel2Mesh_original-pytorch/datasets/data/shapenet/data_tf/02828884/1ac6a3d5c76c8b96edccc47bf0dcf5d3/rendering/01.dat"), 'rb'), encoding="latin1")
pts, normals = data[:, :3], data[:, 3:]

img = io.imread("/root/autodl-tmp/Pixel2Mesh_original-pytorch/datasets/data/shapenet/data_tf/02828884/1ac6a3d5c76c8b96edccc47bf0dcf5d3/rendering/01.png")
img[np.where(img[:, :, 3] == 0)] = 255
img = transform.resize(img, (224, 224))
img = img[:, :, :3].astype(np.float32)

pts -= np.array([0., 0., -0.8])
assert pts.shape[0] == normals.shape[0]
length = pts.shape[0]

img = torch.from_numpy(np.transpose(img, (2, 0, 1)))
normalize_img = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
img_normalized = normalize_img(img)

targets = {
    "images": torch.tensor(img_normalized).unsqueeze(0).to(config["device"]),
    "points": torch.tensor(pts).unsqueeze(0).to(config["device"]),
    "normals": torch.tensor(normals).unsqueeze(0).to(config["device"]),
}

options = {
    "hidden_dim": 192,
    "last_hidden_dim": 192,
    "coord_dim": 3,
    "backbone": "vgg16",
    "gconv_activation": True,
    "z_threshold": 0,
    "align_with_tensorflow": False,
    "optim": edict({"weights": {"normal": 1.6e-4,
        "edge": 0.3,
        "laplace": 0.5,
        "move": 0.1,
        "constant": 1.,
        "chamfer": [1., 1., 1.],
        "chamfer_opposite": 1.,
        "reconst": 0.,} 
    })
}

options = edict(options)

model = P2MModel(options, Ellipsoid([0., 0., -0.8]), [248., 248.], [111.5, 111.5], [0., 0., -0.8])
model.to(config["device"])

loss_criterion = P2MLoss(config["loss"], Ellipsoid([[0., 0., -0.8]]))

loss_criterion.to(config["device"])

# optimizer = torch.optim.Adadelta(model.parameters(), config["learning_rate"])
optimizer = torch.optim.Adam(
                params=list(model.parameters()),
                lr=0.0001,
                betas=(0.9, 0.999),
                weight_decay=1.0e-06
            )


# for i in range(5001):

#     optimizer.zero_grad()
            
#     preds = model(targets["images"])

#     loss, _ = loss_criterion(preds, targets)
#     if i % 10 ==0:
#         print(f"{i}: {loss}")
        
#     if i == 1000:
#         preds1000 = preds
#     if i == 2000:
#         preds2000 = preds
#     if i == 3000:
#         preds3000 = preds
#     if i == 4000:
#         preds4000 = preds
#     if i == 5000:
#         preds5000 = preds
        
#     loss.backward()
#     optimizer.step()



<class 'torch.Tensor'>
torch.Size([3, 224, 224])


/root/miniconda3/envs/p2m/lib/python3.7/site-packages/ipykernel_launcher.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


### 在model基础上继续训练

In [ ]:
optimizer = torch.optim.Adam(
                params=list(model.parameters()),
                lr=0.0001,
                betas=(0.9, 0.999),
                weight_decay=1.0e-06
            )


for i in range(1000):

    optimizer.zero_grad()
            
    preds = model(targets["images"])

    loss, _ = loss_criterion(preds, targets)
    if i % 20 == 0:
        print(f"{i}: {loss}")
    loss.backward()
    optimizer.step()

### 人脸overfitting

In [1]:
from models import backbones
from train import train
import pickle
from pathlib import Path
import io
import numpy as np
import torch
from skimage import io, transform
from models.p2m import P2MModel
from utils.mesh import Ellipsoid
from torchvision.transforms import Normalize
from models.losses.p2m import P2MLoss
from easydict import EasyDict as edict
from utils.visualization import visualize_occupancy, visualize_pointcloud, visualize_mesh


config  = {
    "experiment_name": "template",
    "device": "cuda",
    "learning_rate": 0.0001,
    "batch_size": 8,
    "max_epochs": 100,
    "validate_every_n": 50,
    "print_every_n": 10,
    "loss": edict({"weights": {"normal": 1.6e-4,
        "edge": 0.3,
        "laplace": 0.5,
        "move": 0.1,
        "constant": 1.,
        "chamfer": [1., 1., 1.],
        "chamfer_opposite": 1.,
        "reconst": 0.,} 
    }),
}


data = np.loadtxt("/root/autodl-tmp/Pixel2Mesh_original-pytorch/datasets/data/AFLW2000-3D/AFLW2000/image00002_1.txt")
pts, normals = data[:, :3].astype(np.float32), data[:, 3:].astype(np.float32)

length = pts.shape[0]
choices = np.resize(np.random.permutation(length), 3000)
pts, normals = pts[choices], normals[choices]

pts = pts/1000.

img = io.imread("/root/autodl-tmp/Pixel2Mesh_original-pytorch/datasets/data/AFLW2000-3D/AFLW2000/image00002.jpg")
img = transform.resize(img, (224, 224))
img = img[:, :, :3].astype(np.float32)

pts -= np.array([0., 0., -0.8])
assert pts.shape[0] == normals.shape[0]
length = pts.shape[0]

img = torch.from_numpy(np.transpose(img, (2, 0, 1)))
normalize_img = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
img_normalized = normalize_img(img)

targets = {
    "images": torch.tensor(img_normalized).unsqueeze(0).to(config["device"]),
    "points": torch.tensor(pts).unsqueeze(0).to(config["device"]),
    "normals": torch.tensor(normals).unsqueeze(0).to(config["device"]),
}

options = {
    "hidden_dim": 192,
    "last_hidden_dim": 192,
    "coord_dim": 3,
    "backbone": "vgg16",
    "gconv_activation": True,
    "z_threshold": 0,
    "align_with_tensorflow": False,
    "optim": edict({"weights": {"normal": 1.6e-4,
        "edge": 0.3,
        "laplace": 0.5,
        "move": 0.1,
        "constant": 1.,
        "chamfer": [1., 1., 1.],
        "chamfer_opposite": 1.,
        "reconst": 0.,} 
    })
}

options = edict(options)

model = P2MModel(options, Ellipsoid([0., 0., -0.8]), [248., 248.], [111.5, 111.5], [0., 0., -0.8])
model.to(config["device"])

loss_criterion = P2MLoss(config["loss"], Ellipsoid([[0., 0., -0.8]]))

loss_criterion.to(config["device"])

# optimizer = torch.optim.Adadelta(model.parameters(), config["learning_rate"])
optimizer = torch.optim.Adam(
                params=list(model.parameters()),
                lr=0.0001,
                betas=(0.9, 0.999),
                weight_decay=1.0e-06
            )


for i in range(1000):

    optimizer.zero_grad()
            
    preds = model(targets["images"])

    loss, _ = loss_criterion(preds, targets)
    if i % 10 ==0:
        print(f"{i}: {loss}")
        
    if i == 1000:
        preds1000 = preds
    if i == 2000:
        preds2000 = preds
    if i == 3000:
        preds3000 = preds
    if i == 4000:
        preds4000 = preds
    if i == 5000:
        preds5000 = preds
        
    loss.backward()
    optimizer.step()


/root/miniconda3/envs/p2m/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


<class 'numpy.float32'>
<class 'numpy.float32'>
(224, 224, 3)
<class 'torch.Tensor'>
torch.Size([3, 224, 224])


/root/miniconda3/envs/p2m/lib/python3.7/site-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
THCudaCheck FAIL file=/pytorch/aten/src/THC/THCGeneral.cpp line=383 error=11 : invalid argument


0: 3.6249654293060303
10: 0.4028460681438446
20: 0.07092326134443283
30: 0.03330230340361595
40: 0.02197539247572422
50: 0.0203960370272398
60: 0.02533606067299843
70: 0.019944583997130394
80: 0.020622901618480682
90: 0.028510551899671555
100: 0.02212553471326828
110: 0.028465306386351585
120: 0.040453892201185226
130: 0.02696302719414234
140: 0.0313575342297554
150: 0.02279849536716938
160: 0.02222629450261593
170: 0.028061451390385628
180: 0.03771223872900009
190: 0.026495633646845818
200: 0.018998553976416588
210: 0.027656907215714455
220: 0.015310589224100113
230: 0.01436105277389288
240: 0.012713453732430935
250: 0.011491440236568451
260: 0.010485275648534298
270: 0.010055535472929478
280: 0.009545148350298405
290: 0.00928723718971014
300: 0.009105615317821503
310: 0.00886804424226284
320: 0.008772256784141064
330: 0.008673641830682755
340: 0.008521548472344875
350: 0.00827530212700367
360: 0.008186353370547295
370: 0.00822235457599163
380: 0.008016473613679409
390: 0.008094299584

### 可视化

In [18]:
visualize_pointcloud(preds5000['pred_coord'][0].cpu().detach().numpy(), 10)
visualize_pointcloud(preds5000['pred_coord'][1].cpu().detach().numpy(), 10)
visualize_pointcloud(preds5000['pred_coord'][2].cpu().detach().numpy(), 10)

Output()

Output()

Output()

In [4]:
visualize_pointcloud(pts, .01)

Output()